In [ ]:
# 1. 环境设置和导入

import os
import sys
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# 设置项目根目录
project_root = Path("/mnt/sdb/DiffusionDrive/")  # 回到项目根目录
sys.path.insert(0, str(project_root))

# 检查环境变量
print("Environment Variables:")
print(f"OPENSCENE_DATA_ROOT: {os.environ.get('OPENSCENE_DATA_ROOT', 'NOT SET')}")
print(f"NAVSIM_EXP_ROOT: {os.environ.get('NAVSIM_EXP_ROOT', 'NOT SET')}")

# 导入轨迹预测应用
try:
    from trajectory_app import TrajectoryPredictionApp
    print("✅ 轨迹预测应用导入成功!")
except ImportError as e:
    print(f"❌ 导入失败: {e}")
    print("请确保您在正确的项目根目录下运行此notebook")


In [ ]:
# 2. 应用配置

# 配置选项1: 使用默认配置文件
config_path = project_root / "trajectory_app" / "config" / "default_config.yaml"

# 配置选项2: 直接在代码中定义配置 (推荐用于自定义)
config = {
    "model": {
        "type": "diffusiondrive",  # 或者 "transfuser"
        "checkpoint_path": "/mnt/sdb/DiffusionDrive/bkb/diffusiondrive_navsim_88p1_PDMS",  # 修改为您的模型路径
        "lr": 6e-4
    },
    "data": {
        "navsim_log_path": f"{os.environ.get('OPENSCENE_DATA_ROOT')}/navsim_logs/test",
        "sensor_blobs_path": f"{os.environ.get('OPENSCENE_DATA_ROOT')}/sensor_blobs/test", 
        "cache_path": f"{os.environ.get('NAVSIM_EXP_ROOT')}/metric_cache"
    },
    "visualization": {
        "time_windows": [1.0, 3.0, 6.0],
        "save_formats": ["png"]
    },
    "output": {
        "output_dir": "./tutorial_output"
    },
    "logging": {
        "level": "INFO"
    }
}

print("配置信息:")
print(f"模型类型: {config['model']['type']}")
print(f"模型权重: {config['model']['checkpoint_path']}")
print(f"数据路径: {config['data']['navsim_log_path']}")
print(f"输出目录: {config['output']['output_dir']}")

# 检查关键路径是否存在
data_path = Path(config['data']['navsim_log_path'])
if data_path.exists():
    print(f"✅ 数据路径存在: {data_path}")
else:
    print(f"❌ 数据路径不存在: {data_path}")
    print("   请修改 OPENSCENE_DATA_ROOT 环境变量或配置中的路径")

model_path = Path(config['model']['checkpoint_path']) if config['model']['checkpoint_path'] else None
if model_path and model_path.exists():
    print(f"✅ 模型权重存在: {model_path}")
elif model_path:
    print(f"❌ 模型权重不存在: {model_path}")
    print("   如果没有训练好的模型，应用将使用随机初始化的权重")
else:
    print("⚠️ 未指定模型权重路径，将使用随机初始化")


In [ ]:
# 3. 初始化轨迹预测应用

print("正在初始化轨迹预测应用...")
print("这可能需要几分钟时间来加载模型和数据...")

try:
    # 初始化应用
    app = TrajectoryPredictionApp(config)
    
    # 获取应用信息
    app_info = app.get_app_info()
    
    print("\n" + "="*60)
    print("🎉 应用初始化成功!")
    print("="*60)
    
    print("\n📊 应用状态:")
    print(f"状态: {app_info['status']}")
    
    print(f"\n🤖 模型信息:")
    print(f"类型: {app_info['model']['model_type']}")
    print(f"状态: {app_info['model']['status']}")
    print(f"设备: {app_info['model']['device']}")
    print(f"参数数量: {app_info['model']['num_parameters']:,}")
    
    print(f"\n📁 数据信息:")
    print(f"可用场景: {app_info['data']['total_scenes']:,}")
    print(f"地图位置: {app_info['data']['num_map_locations']}")
    print(f"日志文件: {app_info['data']['num_logs']}")
    print(f"缓存数据: {'是' if app_info['data']['has_metric_cache'] else '否'}")
    
    if app_info['data']['has_metric_cache']:
        print(f"缓存场景: {app_info['data']['metric_cache_scenes']:,}")
    
    print(f"\n🔧 配置信息:")
    print(f"模型类型: {app_info['config']['model_type']}")
    print(f"数据集: {app_info['config']['data_split']}")
    print(f"有权重: {'是' if app_info['config']['has_checkpoint'] else '否'}")
    
except Exception as e:
    print(f"❌ 初始化失败: {e}")
    print("\n请检查:")
    print("1. 环境变量是否正确设置")
    print("2. 数据路径是否存在")
    print("3. 模型权重文件是否存在（如果指定了的话）")
    print("4. 必要的Python包是否已安装")
    raise


In [ ]:
# 4. 单场景轨迹预测示例

# 获取随机场景进行测试
print("获取测试场景...")
test_scenes = app.get_random_scenes(num_scenes=3)
print(f"获取到 {len(test_scenes)} 个测试场景:")
for i, scene_token in enumerate(test_scenes):
    print(f"  {i+1}. {scene_token}")

# 选择第一个场景进行详细分析
selected_scene = test_scenes[0]
print(f"\n选择场景进行分析: {selected_scene}")

# 预测轨迹 (3秒时间窗口)
print("\n正在进行轨迹预测...")
result = app.predict_single_scene(
    scene_token=selected_scene,
    time_window=(0, 3.0),
    save_visualization=True,
    output_dir="./tutorial_output/single_scene"
)

print("\n" + "="*50)
print("🎯 轨迹预测结果")
print("="*50)

# 显示场景信息
metadata = result["scene_metadata"]
print(f"场景令牌: {metadata['token']}")
print(f"场景类型: {metadata['scenario_type']}")
print(f"日志名称: {metadata['log_name']}")
print(f"时间戳: {metadata['timestamp']}")

# 显示预测信息
pred_result = result["prediction_result"]
print(f"\n推理时间: {pred_result['inference_time']:.3f}秒")
print(f"轨迹长度: {pred_result['trajectory_length']} 个点")
if pred_result['time_horizon']:
    print(f"时间跨度: {pred_result['time_horizon']}秒")

# 显示轨迹信息
trajectories = result["trajectories"]["synchronized"]
print(f"\n📈 可用轨迹:")
for traj_name, traj_data in trajectories.items():
    print(f"  • {traj_name}: {len(traj_data['poses'])} 个点, {traj_data['timestamps'][-1]:.1f}秒")

# 显示评估指标
if result["metrics"]:
    metrics = result["metrics"]
    print(f"\n📊 评估指标:")
    print(f"  • ADE (平均位移误差): {metrics['ade']:.2f}m")
    print(f"  • FDE (最终位移误差): {metrics['fde']:.2f}m")
    print(f"  • 最大误差: {metrics['max_error']:.2f}m")
    print(f"  • RMSE: {metrics['rmse']:.2f}m")

# 显示可视化结果
viz_path = result["visualization"]["save_path"]
if viz_path:
    print(f"\n💾 可视化已保存到: {viz_path}")
else:
    print(f"\n📊 显示可视化结果:")

# 显示图像
fig = result["visualization"]["figure"]
plt.show()


In [ ]:
# 5. 不同时间窗口的轨迹可视化比较

print("创建不同时间窗口的轨迹可视化比较...")

# 定义不同的时间窗口
time_windows = [(0, 1.5), (0, 3.0), (0, 6.0)]

# 为同一场景创建不同时间窗口的可视化
results_by_time = {}

for time_window in time_windows:
    print(f"\n生成时间窗口 {time_window[0]:.1f}s - {time_window[1]:.1f}s 的可视化...")
    
    result = app.predict_single_scene(
        scene_token=selected_scene,
        time_window=time_window,
        save_visualization=True,
        output_dir=f"./tutorial_output/time_comparison"
    )
    
    results_by_time[time_window] = result
    
    # 显示当前时间窗口的信息
    trajectories = result["trajectories"]["synchronized"]
    print(f"  时间窗口: {time_window[1]:.1f}s")
    for traj_name, traj_data in trajectories.items():
        filtered_length = len(traj_data['poses'])
        print(f"    • {traj_name}: {filtered_length} 个点")

# 创建时间窗口比较图
print(f"\n创建时间窗口比较图...")
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

for i, time_window in enumerate(time_windows):
    result = results_by_time[time_window]
    scene_data = app.data_manager.load_scene_data(selected_scene)
    trajectories = result["trajectories"]["synchronized"]
    
    # 使用可视化器创建简单的BEV图
    bev_fig = app.visualizer.create_simple_bev_plot(
        scene_data, 
        trajectories, 
        time_window=time_window,
        figsize=(6, 6)
    )
    
    # 复制到子图
    bev_ax = bev_fig.axes[0]
    axes[i].clear()
    
    # 重新渲染到目标axes
    app.visualizer._render_bev_trajectories(axes[i], scene_data, trajectories, time_window)
    
    axes[i].set_title(f'时间窗口: {time_window[1]:.1f}s', fontsize=14, fontweight='bold')
    
    plt.close(bev_fig)  # 关闭临时图

plt.tight_layout()
plt.suptitle(f'时间窗口比较 - 场景: {selected_scene[:12]}...', fontsize=16, y=1.02)
plt.show()

# 显示统计信息
print(f"\n📊 时间窗口比较统计:")
print("-" * 50)
for time_window in time_windows:
    result = results_by_time[time_window]
    if result["metrics"]:
        metrics = result["metrics"]
        print(f"时间窗口 {time_window[1]:.1f}s:")
        print(f"  ADE: {metrics['ade']:.2f}m | FDE: {metrics['fde']:.2f}m | RMSE: {metrics['rmse']:.2f}m")
    else:
        print(f"时间窗口 {time_window[1]:.1f}s: 无评估指标")

print(f"\n💾 所有可视化文件保存到: ./tutorial_output/time_comparison/")


In [ ]:
# 6. 批量场景处理示例

print("批量场景处理示例")
print("这将处理多个场景并生成汇总报告...")

# 获取一批测试场景（这里用5个场景做演示）
batch_scenes = app.get_random_scenes(num_scenes=5)
print(f"\n选择 {len(batch_scenes)} 个场景进行批量处理:")
for i, scene_token in enumerate(batch_scenes):
    print(f"  {i+1}. {scene_token}")

# 批量处理
print(f"\n开始批量处理...")
batch_results = app.predict_batch_scenes(
    scene_tokens=batch_scenes,
    time_window=(0, 3.0),
    output_dir="./tutorial_output/batch_processing",
    max_scenes=5  # 限制最大处理数量
)

print(f"\n" + "="*60)
print("📈 批量处理结果")
print("="*60)

if batch_results:
    # 计算汇总统计
    all_ades = [r["metrics"]["ade"] for r in batch_results if r["metrics"]]
    all_fdes = [r["metrics"]["fde"] for r in batch_results if r["metrics"]]
    processing_times = [r["processing_time"] for r in batch_results]
    
    print(f"✅ 成功处理: {len(batch_results)}/{len(batch_scenes)} 个场景")
    print(f"⏱️ 平均处理时间: {np.mean(processing_times):.2f}s")
    
    if all_ades:
        print(f"\n📊 整体评估指标:")
        print(f"  • 平均 ADE: {np.mean(all_ades):.2f} ± {np.std(all_ades):.2f}m")
        print(f"  • 平均 FDE: {np.mean(all_fdes):.2f} ± {np.std(all_fdes):.2f}m")
        print(f"  • ADE 范围: {np.min(all_ades):.2f}m - {np.max(all_ades):.2f}m")
        print(f"  • FDE 范围: {np.min(all_fdes):.2f}m - {np.max(all_fdes):.2f}m")
    
    # 按地图位置分组统计
    map_stats = {}
    for result in batch_results:
        map_name = result["scene_metadata"].get("map_name", "unknown_map")
        if map_name not in map_stats:
            map_stats[map_name] = {"count": 0, "ades": [], "fdes": []}
        
        map_stats[map_name]["count"] += 1
        if result["metrics"]:
            map_stats[map_name]["ades"].append(result["metrics"]["ade"])
            map_stats[map_name]["fdes"].append(result["metrics"]["fde"])
    
    print(f"\n🎯 按地图位置统计:")
    for map_name, stats in map_stats.items():
        print(f"  • {map_name}: {stats['count']} 个场景")
        if stats["ades"]:
            print(f"    - 平均 ADE: {np.mean(stats['ades']):.2f}m")
            print(f"    - 平均 FDE: {np.mean(stats['fdes']):.2f}m")
    
    # 创建简单的可视化对比
    print(f"\n📊 创建批量结果可视化...")
    
    # 选择前3个成功的结果进行可视化
    viz_results = batch_results[:3]
    
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    for i, result in enumerate(viz_results):
        scene_token = result["scene_token"]
        scene_data = app.data_manager.load_scene_data(scene_token)
        trajectories = result["trajectories"]["synchronized"]
        
        # 渲染BEV视图
        app.visualizer._render_bev_trajectories(
            axes[i], scene_data, trajectories, (0, 3.0)
        )
        
        # 设置标题
        map_name = result["scene_metadata"].get("map_name", "unknown")
        if result["metrics"]:
            ade = result["metrics"]["ade"]
            axes[i].set_title(f'{map_name}\nADE: {ade:.2f}m', fontsize=12)
        else:
            axes[i].set_title(f'{map_name}', fontsize=12)
    
    plt.tight_layout()
    plt.suptitle('批量处理结果示例 (前3个场景)', fontsize=16, y=1.02)
    plt.show()
    
else:
    print("❌ 批量处理失败或无结果")

print(f"\n💾 批量处理结果保存到: ./tutorial_output/batch_processing/")
print(f"📄 查看 batch_summary.yaml 文件获取详细的汇总报告")


In [ ]:
# 7. 创建演示可视化

print("创建综合演示可视化...")
print("这将创建多个场景和时间窗口的演示材料")

# 创建演示
demo_result = app.create_demo_visualization(
    num_scenes=2,  # 2个场景
    time_windows=[(0, 1.5), (0, 3.0), (0, 6.0)],  # 3个时间窗口
    output_dir="./tutorial_output/demo"
)

print(f"\n" + "="*60)
print("🎬 演示可视化创建完成")
print("="*60)

demo_results = demo_result["results"]
output_dir = demo_result["output_dir"]

print(f"✅ 创建了 {len(demo_results)} 个演示可视化")
print(f"📁 保存路径: {output_dir}")

# 按场景和时间窗口统计
scene_count = len(set(r["scene_token"] for r in demo_results))
time_windows = list(set(r["time_window"] for r in demo_results))

print(f"\n📊 演示统计:")
print(f"  • 场景数量: {scene_count}")
print(f"  • 时间窗口: {time_windows}")
print(f"  • 总可视化: {len(demo_results)}")

# 显示每个演示的详细信息
print(f"\n📋 演示详情:")
for result in demo_results:
    scene_idx = result["scene_index"]
    time_window = result["time_window"]
    scene_token = result["scene_token"][:12]
    viz_path = result["result"]["visualization"]["save_path"]
    
    if result["result"]["metrics"]:
        ade = result["result"]["metrics"]["ade"]
        print(f"  场景 {scene_idx} | 时间 {time_window[1]:.1f}s | Token: {scene_token}... | ADE: {ade:.2f}m")
    else:
        print(f"  场景 {scene_idx} | 时间 {time_window[1]:.1f}s | Token: {scene_token}... | 无指标")
    
    if viz_path:
        print(f"    📄 {viz_path.name}")

print(f"\n💡 使用建议:")
print(f"  1. 查看 {output_dir} 目录下的所有生成图像")
print(f"  2. 比较不同时间窗口下的轨迹预测效果")
print(f"  3. 分析不同场景类型的模型表现")
print(f"  4. 使用这些可视化作为报告或演示材料")
